In [26]:
import fastfusion
from fastfusion.mapper import Metrics
import time

spec = fastfusion.Specification.from_yaml(
    "architectures/fastfusion/tpu_like.yaml",
    "workloads/fastfusion/gemm_16k.yaml",
)
spec.mapper.ffm.metrics = Metrics.ENERGY | Metrics.LATENCY
start = time.time()
pmappings = fastfusion.mapper.FFM.make_pmappings(spec)
mappings = fastfusion.mapper.FFM.join_pmappings(spec, pmappings)
end = time.time()
duration = end - start
print(f"Mapper time: {duration:.2}")

WARNING Loading configuration file from /home/gilbertm/work/venv/fastfusion/config.yaml
Generating pmapping templates for compute scalar_unit Einsum Matmul: 0it [00:00, ?it/s]           | 0/2 [00:00<?, ?it/s]
Generating pmapping templates for compute MAC Einsum Matmul: 8it [00:00, 86.81it/s]       | 1/2 [00:00<00:00,  1.05it/s]
Generating jobs: 100%|████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.92it/s]
WARNING Einsum Matmul has 8 pmapping templates:
WARNING 	[WA,I,A in MainMemory] T-m,na initial=symbol stride=symbol [A in GlobalBuffer] T-ni initial=symbol stride=symbol [I in GlobalBuffer] T-na initial=symbol stride=symbol S-Z-m initial=symbol stride=symbol S-Y-ni initial=symbol stride=symbol S-X-na initial=symbol stride=symbol [WA in Register] T-m initial=symbol stride=symbol MAC computes Matmul
WARNING 	[WA,I,A in MainMemory] T-m,ni initial=symbol stride=symbol [I in GlobalBuffer] T-na initial=symbol stride=symbol [A in GlobalBuffer] T-ni i

Einsum Matmul has 12 pmappings with 8 compatibilities


Compressing pmappings: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 197.31it/s]


Not tracking GlobalBuffer because it is never reserved for multiple pmappings.
Not tracking LocalBuffer because it is never reserved for multiple pmappings.
Not tracking MainMemory because it is never reserved for multiple pmappings.
Not tracking Register because it is never reserved for multiple pmappings.


Grouping pmappings: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 314.84it/s]

Mapper time: 8.6


In [27]:
chosen_mapping = mappings.data.iloc[0]['Total<SEP>mapping']

In [31]:
mappings.data

,Total<SEP>latency,Total<SEP>energy,Matmul<SEP>stride0,Matmul<SEP>stride1,Matmul<SEP>stride2,Matmul<SEP>stride3,Matmul<SEP>stride4,Matmul<SEP>latency<SEP>compute,Matmul<SEP>latency<SEP>Register,Matmul<SEP>latency<SEP>GlobalBuffer,...,Matmul<SEP>energy<SEP>MainMemory<SEP>write,Matmul<SEP>energy<SEP>MAC<SEP>compute,Matmul<SEP>energy<SEP>MainMemory<SEP>leak,Matmul<SEP>energy<SEP>GlobalBuffer<SEP>leak,Matmul<SEP>energy<SEP>LocalBuffer<SEP>leak,Matmul<SEP>energy<SEP>scalar_unit<SEP>leak,Matmul<SEP>energy<SEP>Register<SEP>leak,Matmul<SEP>energy<SEP>MAC<SEP>leak,Matmul<SEP>mapping,Total<SEP>mapping
1,0.075497,1.274725,512,1024,128,128,128,0.067109,0,0.075497,...,0.015097,0.307863,0,0,0,0,0,0,"type=None nodes=[Storage(type=None, tensors=['...",<fastfusion.mapper.FFM._interface.main.Mapping...


In [30]:
import csv

with open("ffm_gemm_16k.csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["mapper_time", "energy", "latency"])
    writer.writerow([duration, mappings.data.iloc[0]["Total<SEP>energy"], mappings.data.iloc[0]["Total<SEP>latency"]])